## Análise estatística
### Regressão Logística

Vamos fazer um exercício de Regressão Logística, trabalhando com a base de dados do Titanic.
Utilizaremos os arquivos disponibilizados no Moodle que foram preparados para esta atividade. Embora os nomes sejam os mesmos, estes arquivos do Moodle poder ser diferentes (em conteúdo) daqueles encontrados no site Kaggle.
A base de dados (treinamento) é uma coleção de dados sobre alguns dos passageiros (891 ou 892) e o objetivo é prever a sobrevivência (1 se o passageiro sobreviveu ou 0 caso contrário), baseado em algumas características tais como classe de serviço, sexo, idade, etc. A base de dados de teste tem a mesma estrutura da base de treinamento, mas a variável Survived não contém nenhum valor (ou não está presente)
Depois de construir seu modelo, você vai aplicá-lo ao este conjunto de dados de teste e prever o valor da variável Survived.
Como se pode ver, utilizaremos tanto variáveis categóricas como contínuas.
Quando trabalhamos com uma base de dados real, precisamos levar em conta o fato de podermos ter dados faltantes ou corrompidos, e, portanto, precisamos preparar a base de dados para nossa análise.
As tarefas a serem realizadas são:

a)       Fazer a preparação dos dados para serem utilizados na análise, considerando que serão utilizadas apenas as variáveis Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Embarked

In [ ]:
#install.packages("dplyr")
#install.packages("Hmisc")

In [2]:
# Declarando as bibliotecas
library("dplyr")
library("Hmisc")

In [3]:
# Carregando os dados
treino = read.csv('titanic_dw_train.csv')
teste  = read.csv('titanic_dw_test.csv')

In [4]:
# Verificando a estrutura dos dados
str(treino)

'data.frame':	891 obs. of  12 variables:
 $ PassengerId: int  2 3 4 5 6 10 11 12 13 15 ...
 $ Survived   : int  0 1 0 1 0 0 1 0 0 1 ...
 $ Pclass     : int  3 1 2 1 3 3 1 3 3 2 ...
 $ Name       : Factor w/ 890 levels "Abbing, Mr. Anthony",..: 424 561 389 160 187 872 234 13 63 354 ...
 $ Sex        : Factor w/ 2 levels "female","male": 2 1 2 1 2 2 2 2 2 2 ...
 $ Age        : num  29 37 36 NA 30 21 53 20 26 62 ...
 $ SibSp      : int  0 1 0 0 0 1 1 0 0 0 ...
 $ Parch      : int  0 0 0 1 0 0 1 0 0 0 ...
 $ Ticket     : Factor w/ 683 levels "110152","110413",..: 346 98 149 31 661 252 298 656 389 633 ...
 $ Fare       : num  7.85 90 13 55 8.05 ...
 $ Cabin      : Factor w/ 144 levels "A10","A11","A14",..: NA 77 NA 122 NA NA 11 NA NA NA ...
 $ Embarked   : Factor w/ 3 levels "C","Q","S": 3 2 3 3 3 3 3 3 3 3 ...


In [5]:
summary(treino)

  PassengerId        Survived          Pclass     
 Min.   :   2.0   Min.   :0.0000   Min.   :1.000  
 1st Qu.: 338.5   1st Qu.:0.0000   1st Qu.:2.000  
 Median : 667.0   Median :0.0000   Median :3.000  
 Mean   : 656.1   Mean   :0.3771   Mean   :2.293  
 3rd Qu.: 979.5   3rd Qu.:1.0000   3rd Qu.:3.000  
 Max.   :1304.0   Max.   :1.0000   Max.   :3.000  
                                                  
                            Name         Sex           Age         
 Kelly, Mr. James             :  2   female:318   Min.   : 0.1667  
 Abbing, Mr. Anthony          :  1   male  :573   1st Qu.:21.0000  
 Abbott, Master. Eugene Joseph:  1                Median :28.0000  
 Abbott, Mr. Rossmore Edward  :  1                Mean   :29.9936  
 Abelseth, Miss. Karen Marie  :  1                3rd Qu.:39.0000  
 Abelseth, Mr. Olaus Jorgensen:  1                Max.   :80.0000  
 (Other)                      :884                NA's   :172      
     SibSp            Parch            Ticket   

In [6]:
# Verificando valores únicos e dados faltantes para cada variável
descricao <- describe(treino)
descricao

treino 

 12  Variables      891  Observations
--------------------------------------------------------------------------------
PassengerId 
       n  missing distinct     Info     Mean      Gmd      .05      .10 
     891        0      891        1    656.1    434.5     68.5    125.0 
     .25      .50      .75      .90      .95 
   338.5    667.0    979.5   1171.0   1241.5 

lowest :    2    3    4    5    6, highest: 1297 1298 1299 1303 1304
--------------------------------------------------------------------------------
Survived 
       n  missing distinct     Info      Sum     Mean      Gmd 
     891        0        2    0.705      336   0.3771   0.4703 

--------------------------------------------------------------------------------
Pclass 
       n  missing distinct     Info     Mean      Gmd 
     891        0        3    0.818    2.293   0.8697 
                            
Value          1     2     3
Frequency    220   190   481
Proportion 0.247 0.213 0.540
----------------

In [7]:
# Temos valores ausentes nas variáveis Age, Fare, Cabin e Embarked

i) Do conjunto de dados original, você deve selecionar um subconjunto apenas com as variáveis indicadas acima e a variável PassengerId.

In [8]:
dados <- subset(treino, select=c(Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Embarked, PassengerId))
head(dados)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,PassengerId
,<int>,<int>,<fct>,<dbl>,<int>,<int>,<dbl>,<fct>,<int>
1,0,3,male,29,0,0,7.8542,S,2
2,1,1,female,37,1,0,90.0000,Q,3
3,0,2,male,36,0,0,13.0000,S,4
4,1,1,female,NA,0,1,55.0000,S,5
5,0,3,male,30,0,0,8.0500,S,6
6,0,3,male,21,1,0,6.4958,S,10


ii) Você deve  atribuir um valor para os NAs na variável Age. Utilize algum critério razoável, por exemplo, o valor médio.

In [9]:
# Contagem de valores NA em Age
sum(is.na(dados$Age))

[1] 172

In [10]:
# Calculando valor médio das idades
idadeMedia=mean(treino$Age, na.rm=TRUE)
idadeMedia=round(idadeMedia, digits=0)
idadeMedia

[1] 30

In [11]:
# Substituindo campos NA em Age pelo valor médio das idades
dados2 <- dados %>% mutate(Age = ifelse(is.na(Age), idadeMedia, Age))
dados2$Age

[1] 29.0000 37.0000 36.0000 30.0000 30.0000 21.0000 53.0000 20.0000 26.0000
 [10] 62.0000 24.0000 38.0000 27.0000 30.0000  1.0000 24.0000 33.0000 29.0000
 [19] 48.0000 14.0000 27.0000 18.0000 23.0000 30.0000 30.0000 32.0000 30.0000
 [28] 26.0000 27.0000 74.0000 28.0000 30.0000 36.5000 46.0000 30.0000 25.0000
 [37] 19.0000 30.0000 30.0000 27.0000 29.0000 23.0000 30.0000 24.0000 24.0000
 [46] 21.0000 30.0000  1.0000 30.0000 53.0000 17.0000 30.0000 24.0000 30.0000
 [55] 22.0000 29.0000 30.0000 28.0000 28.0000 30.0000 21.0000 42.0000 30.0000
 [64] 60.0000 30.0000 61.0000 23.0000 21.0000 20.0000 23.0000 22.0000 30.0000
 [73] 29.0000 30.0000 30.0000 30.0000 24.0000 51.0000 35.0000 48.0000 47.0000
 [82] 26.0000 21.0000 54.0000 30.0000 30.0000 30.0000 24.0000 29.0000 30.0000
 [91] 26.0000 64.0000 49.0000 39.0000 45.5000 42.0000 39.0000 20.0000 22.0000
[100] 18.0000 18.0000 10.0000 39.0000 19.0000  2.0000 25.0000 33.0000 34.0000
[109] 38.0000 20.0000 15.0000 30.0000 52.0000  4.0000 30.0000 50.0000 31.0000
[118] 16.0000 32.0000 31.0000 30.0000 18.0000 25.0000 45.5000 40.0000 28.0000
[127] 22.0000 45.0000 30.0000 39.0000 30.0000  7.0000 27.0000 71.0000 60.5000
[136] 51.0000 24.0000 32.0000 25.0000 38.0000 11.5000 30.0000 29.0000 21.0000
[145] 26.0000 50.0000 22.0000  3.0000 30.0000 40.0000 30.0000 30.0000 49.0000
[154] 34.0000 30.0000 49.0000 28.0000 40.0000 30.0000  6.0000 24.0000 30.0000
[163] 29.0000 26.0000 36.0000 38.0000 24.0000 32.0000 18.0000 30.0000 18.0000
[172] 16.0000 24.0000 35.0000 18.0000  1.0000 39.0000 27.0000  3.0000 14.0000
[181] 30.0000 33.0000 23.0000 28.0000 31.0000 30.0000 18.0000 17.0000 22.0000
[190] 26.0000 30.0000 45.0000 23.0000 23.0000 26.0000  5.0000 18.0000 62.0000
[199] 27.0000 18.5000 55.0000  0.9167 21.0000 19.0000 45.0000 30.0000 25.0000
[208] 30.0000 30.0000 41.0000 36.0000 30.0000 17.0000 37.0000 33.0000 22.0000
[217] 30.0000 23.0000 22.0000 31.0000  4.0000 18.0000 56.0000 28.0000 16.0000
[226]  9.0000 29.0000 17.0000 25.0000 28.0000 30.0000 31.0000 35.0000 30.0000
[235] 30.0000 30.0000 26.0000 30.0000 70.5000  7.0000 27.0000 18.0000 22.0000
[244] 16.0000 22.0000 29.0000 30.0000 22.0000 36.0000 47.0000 34.0000 50.0000
[253] 30.0000 32.0000 60.0000 27.0000 30.0000 30.0000 27.0000 30.0000 18.0000
[262] 55.5000 48.0000  5.0000 21.0000 30.0000  3.0000 27.0000 32.0000 16.0000
[271] 54.0000 30.0000 30.0000 41.0000 34.0000 21.0000 45.0000 30.0000 25.0000
[280] 33.0000 35.0000 65.0000 20.0000  2.0000 32.0000 36.0000 22.0000 30.0000
[289] 39.0000  1.0000 40.0000 34.0000  4.0000 30.5000 35.0000 30.0000 19.0000
[298] 30.0000 23.0000 30.0000 32.5000 30.0000 23.0000 30.0000 19.0000 30.0000
[307]  1.0000 30.0000 29.0000 33.0000 30.0000 30.0000 30.0000 39.0000  6.0000
[316] 62.0000 32.0000 38.0000  2.0000 29.0000 21.0000 15.0000 29.0000 21.0000
[325] 30.0000 23.0000 16.0000 30.0000 21.0000 33.0000 48.0000 16.0000 21.0000
[334] 19.0000 17.0000 24.0000 47.0000 26.0000 47.0000 25.0000  6.0000 45.0000
[343] 30.0000 30.0000  2.0000 29.0000 18.0000 22.0000 47.0000 28.0000 28.0000
[352] 50.0000 26.0000 15.0000 24.0000 32.0000 31.0000  6.0000 30.0000 18.0000
[361] 43.0000 33.0000 29.0000 59.0000 42.0000 17.0000 30.0000 30.0000 30.0000
[370] 30.0000 38.0000 30.0000 42.0000 48.0000 30.0000 64.0000 18.0000 54.0000
[379] 22.0000  2.0000  0.6667 24.0000 30.0000 23.0000 30.0000 29.0000  0.7500
[388] 30.0000 60.0000 47.0000 45.0000 30.0000 30.0000 26.0000 63.0000 35.0000
[397] 37.0000 22.0000 26.0000 30.0000 29.0000  8.0000 43.0000 18.0000 25.0000
[406] 26.0000 30.0000 50.0000 29.0000 23.5000 44.0000 30.0000 66.0000 30.0000
[415] 30.0000 30.0000 24.0000 30.0000 55.0000 67.0000 65.0000 19.0000  4.0000
[424] 45.0000 30.0000 42.0000 14.0000 18.5000 34.0000 17.0000 30.0000 42.0000
[433] 27.0000 12.0000 49.0000 35.0000 30.0000 28.0000 30.0000 30.0000 60.0000
[442] 26.0000 26.0000 21.0000 25.0000 26.0000 30.0000 39.0000 30.0000 36.0000
[451] 32.0000 55.0000 30.0000 30.0000 22.0000 30.0000 21.0000 71.0000 28.0000
[460] 30.0000 30.0000 26

In [12]:
# Contagem de valores NA em Age
sum(is.na(dados2$Age))

[1] 0

In [13]:
# Verificando valores NA no arquivo de teste
sum(is.na(teste$Age))

[1] 91

In [14]:
# Substituindo campos NA em Age pelo valor médio das idades no arquivo de teste
teste <- teste %>% mutate(Age = ifelse(is.na(Age), idadeMedia, Age))
teste$Age

[1] 41.0000 26.0000 54.0000 48.0000 62.0000 36.0000 21.0000 30.0000 45.0000
 [10] 22.0000 17.0000 28.0000  0.9167 30.0000 41.0000  9.0000 45.0000 34.0000
 [19] 19.0000 30.0000 46.0000 38.0000 30.0000 28.5000  9.0000  3.0000 18.0000
 [28] 30.0000 36.0000 24.0000 29.0000 27.0000 14.0000 30.0000 39.0000 30.0000
 [37] 34.0000 36.0000 27.0000 45.0000 28.0000 40.0000 24.0000 30.0000 21.0000
 [46] 36.0000 21.0000 10.0000 21.0000  0.7500 30.0000 47.0000 35.0000 24.0000
 [55] 30.0000 24.0000 34.0000 19.0000 57.0000  8.0000 31.0000  1.0000 27.0000
 [64] 30.0000 48.0000 22.0000 30.0000  9.0000 22.0000 30.0000 30.0000 30.0000
 [73] 18.0000 40.0000 29.0000 13.0000 32.0000 31.0000 44.0000 30.0000 25.0000
 [82]  9.0000 16.0000 28.0000 51.0000 30.0000 25.0000 30.0000 15.0000 17.0000
 [91] 27.0000 17.0000 49.0000 19.0000 30.0000 22.0000 63.0000 30.0000 24.0000
[100] 22.0000 30.0000 63.0000 30.0000 21.0000 30.0000 21.0000 30.0000 30.0000
[109] 45.0000 30.0000 30.0000 16.0000 25.0000 30.0000 53.0000  2.0000 20.0000
[118] 30.0000 30.0000 35.0000 18.0000 30.0000 42.0000 36.0000 33.0000 24.5000
[127] 30.0000 16.0000 43.0000 38.0000 40.0000 45.0000 33.0000 20.0000  4.0000
[136] 50.0000 36.0000 30.0000 30.0000 30.0000 46.0000 29.0000 50.0000 62.0000
[145] 21.0000 32.0000 10.0000 30.0000 41.0000 16.0000 51.0000 38.0000 39.0000
[154] 20.0000 50.0000 25.0000 30.0000 30.0000 25.0000 30.0000 27.0000 30.0000
[163] 29.0000 58.0000  4.0000 30.0000 26.0000 30.0000 19.0000 26.5000 30.0000
[172]  6.0000 20.0000 39.0000 30.0000 45.0000 40.0000 30.0000 32.5000 61.0000
[181]  1.0000 24.0000 27.0000 40.0000 28.0000 42.0000 18.0000 20.0000 47.0000
[190] 20.0000 36.0000 40.0000 16.0000 23.0000 24.0000 30.0000 31.0000 40.5000
[199] 26.0000 54.0000 30.0000 30.0000 30.0000 32.0000 30.0000  3.0000 37.0000
[208] 18.0000 24.0000 20.0000 30.0000 65.0000 37.0000 50.0000 60.0000 30.0000
[217] 30.0000 30.0000 31.0000 57.0000 30.0000 30.0000 36.0000 38.0000 30.0000
[226] 28.0000 30.0000 30.0000 30.0000 40.0000 35.0000 27.0000 28.0000 38.0000
[235]  8.0000 46.0000 30.0000 30.0000 45.0000 30.0000 24.0000 23.0000  2.0000
[244] 57.0000 39.0000 25.0000 37.0000 30.0000 48.0000  6.0000 51.0000 30.0000
[253] 30.0000 31.0000 25.0000 41.0000 32.0000 30.0000 30.0000 17.0000 30.0000
[262] 48.0000 22.0000 30.0000 30.0000 20.0000 28.0000 29.0000 24.0000 22.0000
[271] 25.0000 30.0000 28.0000 19.0000 35.0000 17.0000 19.0000 38.0000 20.0000
[280] 17.0000 25.0000 23.0000 31.0000 23.0000 31.0000 30.0000 30.0000 34.0000
[289] 37.0000 30.0000 24.0000 20.5000 17.0000 34.0000 18.0000 42.0000 24.0000
[298] 30.0000 13.0000 59.0000 52.0000 21.0000 60.0000 30.0000 30.0000 14.0000
[307] 30.0000 24.0000 23.0000 30.0000 36.0000 40.5000 19.0000 28.0000 30.0000
[316] 19.0000 45.0000 41.0000 32.0000 32.0000 36.0000 27.0000 30.0000 27.0000
[325] 18.0000 10.0000 12.0000 25.0000 31.0000 30.0000 30.0000 64.0000 31.0000
[334] 23.0000 30.0000 16.0000 30.0000 17.0000 33.0000 30.0000 30.0000 33.0000
[343] 31.0000 19.0000 18.0000 18.0000 36.0000 30.0000 30.0000 22.0000 25.0000
[352] 21.0000 30.0000 25.0000 41.0000 30.0000 44.0000 30.0000 24.0000  2.0000
[361] 30.0000 30.0000 30.0000 30.0000 25.0000 23.0000 47.0000 30.0000  0.7500
[370] 32.5000 30.0000 22.0000 30.0000 21.0000 33.0000 33.0000 34.0000 44.0000
[379] 37.0000 21.0000  9.0000 19.0000 48.0000 28.5000 56.0000 22.0000 31.0000
[388] 22.5000 45.0000 41.0000 27.0000 18.0000 24.0000 30.0000 20.0000 48.0000
[397] 33.0000 30.0000 57.0000 30.0000 61.0000 17.0000 21.0000 17.0000 35.0000
[406] 39.0000 30.0000 22.0000 30.0000 27.0000  0.8333 30.0000 28.0000 30.0000
[415] 28.0000 21.0000 39.0000 16.0000

In [15]:
# Verificando se a correção foi bem sucedida
sum(is.na(teste$Age))

[1] 0

iii) Você deve remover as linhas onde ainda estiverem faltando dados, depois de atribuir o valor para os NAs de Age. Poucas linhas estarão ainda com dados faltantes.

In [16]:
# Checando variáveis com dados nulos
sapply(dados2, function(x) sum(is.na(x)))

Survived      Pclass         Sex         Age       SibSp       Parch 
          0           0           0           0           0           0 
       Fare    Embarked PassengerId 
          1           1           0

In [17]:
# Removendo linhas com dados NA
dados3 <- na.omit(dados2)
sapply(dados3, function(x) sum(is.na(x)))

Survived      Pclass         Sex         Age       SibSp       Parch 
          0           0           0           0           0           0 
       Fare    Embarked PassengerId 
          0           0           0

In [18]:
# Checando variáveis com dados nulos no arquivo de teste
sapply(teste, function(x) sum(is.na(x)))

PassengerId      Pclass        Name         Sex         Age       SibSp 
          0           0           0           0           0           0 
      Parch      Ticket        Fare       Cabin    Embarked 
          0           0           0         325           1

In [19]:
# Removendo linhas com valores NA na coluna Embarked
teste <- teste[!is.na(teste$Embarked), ]

In [20]:
# Checando resultado da operação
sapply(teste, function(x) sum(is.na(x)))

PassengerId      Pclass        Name         Sex         Age       SibSp 
          0           0           0           0           0           0 
      Parch      Ticket        Fare       Cabin    Embarked 
          0           0           0         325           0

b) Você deve criar um modelo  onde Survived será uma função das demais variáveis.

In [21]:
modelo = glm(Survived ~ Pclass + Sex + Age + SibSp + Parch +
               Fare + Embarked, family = binomial(link = "logit"), data = dados3)
summary(modelo)


Call:
glm(formula = Survived ~ Pclass + Sex + Age + SibSp + Parch + 
    Fare + Embarked, family = binomial(link = "logit"), data = dados3)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4943  -0.6638  -0.4369   0.6726   2.3739  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  4.559315   0.534618   8.528  < 2e-16 ***
Pclass      -0.954490   0.140704  -6.784 1.17e-11 ***
Sexmale     -2.643916   0.197453 -13.390  < 2e-16 ***
Age         -0.030372   0.007308  -4.156 3.24e-05 ***
SibSp       -0.234982   0.102628  -2.290   0.0220 *  
Parch       -0.085400   0.100582  -0.849   0.3958    
Fare         0.002566   0.002218   1.157   0.2473    
EmbarkedQ   -0.431404   0.360727  -1.196   0.2317    
EmbarkedS   -0.462008   0.224688  -2.056   0.0398 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1177.91  on 888  degrees of freedom
Residual 

c) Faça as análises do modelo, verificando a significância estatística das variáveis (e seus parâmetros ajustados), gráficos diagnósticos, etc.

In [22]:
# As variáveis Parch e Fare possuem um p-value muito alto, indicando que elas não tem impacto
# na variável resposta. Vamos retirar elas do modelo e reavaliar o modelo.

d) Atualize o modelo como consequência da análise realizada no item anterior.

In [23]:
# Removendo a variável Parch do modelo
modelo2 = glm(Survived ~ Pclass + Sex + Age + SibSp + 
               Fare + Embarked, family = binomial(link = "logit"), data = dados3)
summary(modelo2)


Call:
glm(formula = Survived ~ Pclass + Sex + Age + SibSp + Fare + 
    Embarked, family = binomial(link = "logit"), data = dados3)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5159  -0.6615  -0.4318   0.6724   2.3835  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  4.566726   0.533167   8.565  < 2e-16 ***
Pclass      -0.972799   0.138793  -7.009  2.4e-12 ***
Sexmale     -2.607096   0.191827 -13.591  < 2e-16 ***
Age         -0.030318   0.007303  -4.152  3.3e-05 ***
SibSp       -0.253371   0.100743  -2.515   0.0119 *  
Fare         0.002084   0.002098   0.993   0.3206    
EmbarkedQ   -0.400263   0.358072  -1.118   0.2636    
EmbarkedS   -0.473995   0.224173  -2.114   0.0345 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1177.91  on 888  degrees of freedom
Residual deviance:  814.91  on 881  degrees of freedom
AIC: 830.91

Num

In [24]:
# Conseguimos manter os índices próximos aos da análise original com a
# retirada da variável Parch, com uma pequena redução no AIC

In [25]:
# A variável Fare também possui um p-value muito alto, vamos retirá-la do modelo
modelo3 = glm(Survived ~ Pclass + Sex + Age + SibSp + 
               Embarked, family = binomial(link = "logit"), data = dados3)
summary(modelo3)


Call:
glm(formula = Survived ~ Pclass + Sex + Age + SibSp + Embarked, 
    family = binomial(link = "logit"), data = dados3)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4571  -0.6537  -0.4264   0.6658   2.3948  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  4.817343   0.474614  10.150  < 2e-16 ***
Pclass      -1.039165   0.122479  -8.484  < 2e-16 ***
Sexmale     -2.620034   0.191419 -13.687  < 2e-16 ***
Age         -0.030470   0.007289  -4.180 2.91e-05 ***
SibSp       -0.230943   0.097621  -2.366   0.0180 *  
EmbarkedQ   -0.430583   0.356818  -1.207   0.2275    
EmbarkedS   -0.517705   0.219652  -2.357   0.0184 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1177.91  on 888  degrees of freedom
Residual deviance:  815.96  on 882  degrees of freedom
AIC: 829.96

Number of Fisher Scoring iterations: 5


In [26]:
# Conseguimos manter os índices próximos aos da análise original com a
# retirada da variável Fare, com uma pequena redução no AIC

In [27]:
# Avaliando a super dispersão do modelo
deviance(modelo3)/df.residual(modelo3)

[1] 0.9251252

In [28]:
# Valor próximo de 1, não temos superdispersão nos dados

e) Faça as previsões da variável Survived na base de dados de teste utilizando o modelo refinado, e prepare um arquivo CSV para submissão que contenha apenas duas colunas: PassengerId, Survived.

In [29]:
results.Survived = predict(modelo3, newdata = subset(teste, select = c(2, 4, 5, 6, 7, 9, 11)),
                           type = "response")
results.Survived = ifelse(results.Survived > 0.5, 1, 0)
table(results.Survived)
results.Survived

results.Survived
  0   1 
263 154 

1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
  0   0   1   1   0   1   0   0   0   0   1   0   1   0   0   1   1   0   0   0 
 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40 
  0   0   0   0   0   1   1   0   0   0   0   1   1   0   1   0   1   1   1   0 
 41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60 
  1   1   0   0   1   1   1   0   0   1   0   0   1   1   0   0   1   0   1   0 
 61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80 
  0   0   1   1   0   0   0   0   1   0   0   0   1   1   0   1   1   1   0   0 
 81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 
  0   0   0   0   0   1   0   1   0   1   0   0   1   0   1   0   1   0   0   1 
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 
  0   1   1   0   0   0   0   0   1   0   0   1   0   1   1   0   0   0   0   1 
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 
  0   0   0   0   1   0   1   1   1   0   0   1   1   1   0   0   1   1   0   0 
141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 
  0   1   1   0   1   0   1   0   1   0   0   0   0   1   1   0   0   1   0   1 
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 
  1   1   0   1   1   0   1   0   1   0   0   0   0   1   0   1   1   0   1   0 
181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 
  1   0   0   0   0   0   1   0   0   1   0   0   0   1   0   0   1   0   0   1 
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 
  0   1   1   0   0   1   0   0   0   0   0   0   0   0   1   0   0   0   1   0 
221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 
  0   0   1   0   0   0   1   0   1   1   1   0   0   0   0   0   0   0   0   1 
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 
  0   1   1   0   1   0   1   0   0   0   0   1   0   0   0   0   0   0   1   1 
261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 279 280 281 
  0   1   0   1   0   0   1   1   0   1   1   0   0   0   1   0   1   0   0   1 
282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 
  0   1   0   1   0   0   0   0   1   0   0   0   0   1   0   0   1   0   1   0 
302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 
  0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   1 
322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 
  0   0   1   0   0   1   1   0   1   0   1   0   0   1   0   0   0   0   0   0 
342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 
  1   0   1   1   0   0   1   0   0   1   1   1   1   0   1   1   0   1   0   0 
362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 
  0   1   1   0   1   0   0   0   0   1   1   0   0   0   0   0   0   1   0   1 
382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 
  0   0   1   1   1   0   0   1   0   1   1   1   0   0   1   0   1   0   0   0 
402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 
  1   0   1   0   0   0   1   1   1   0   0   0   0   0   0   0   0

f) Submeta seu arquivo e também o script R com todas as análises realizadas, até a criação do arquivo CSV.

In [30]:
submissao = data.frame(PassengerId = teste[, "PassengerId"],
                        Survived = results.Survived)

write.csv(submissao, "titanic-submissao.csv",
          row.names = FALSE, quote = FALSE)